In [1]:
import pandas as pd
import ast

In [2]:
movies_dataset_df = pd.read_parquet('../data/movies_data/movies_dataset.parquet')

In [3]:
movies_dataset_df.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [36]:
production_countries = movies_dataset_df[['production_countries','id']]

In [37]:
production_countries = production_countries.rename(columns={'id': 'movie_id'})

In [38]:
production_countries.iloc[0]['production_countries']

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [6]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   production_countries  45463 non-null  object
 1   movie_id              45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [7]:
production_countries = production_countries.dropna(subset=['production_countries'])

In [8]:
production_countries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   production_countries  45463 non-null  object
 1   movie_id              45463 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


In [21]:
production_countries['production_countries'] = production_countries['production_countries'].astype('str')
production_countries['movie_id'] = production_countries['movie_id'].astype('str')

In [32]:
production_countries_list_dict = []

for idx, row in production_countries.iterrows():
    try:
        if isinstance(row['production_countries'], str) and row['production_countries']:
            production_countries_dicts = ast.literal_eval(row['production_countries'])
            for production_countries_dict in production_countries_dicts:
                production_countries_list_dict.append({**production_countries_dict, 'movie_id': row['movie_id']})
    except Exception as e:
        print(f"Error en el índice {idx}: {e}")

In [29]:
production_countries.iloc[19729]['production_countries']

'6.0'

In [10]:
# Filtrar las filas donde la columna 'A' no contiene los strings 'True' o 'False'
production_countries = production_countries[~production_countries['production_countries'].isin(['True', 'False'])]

In [23]:
production_countries = production_countries[~production_countries.applymap(lambda x: isinstance(x, str) and x in ['True', 'False']).any(axis=1)]

C:\Users\franc\AppData\Local\Temp\ipykernel_18672\3090105200.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  production_countries = production_countries[~production_countries.applymap(lambda x: isinstance(x, str) and x in ['True', 'False']).any(axis=1)]


In [31]:
# Función para verificar si una cadena tiene la forma de un float
def is_float_string(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

# Filtrar las filas donde la columna 'A' no contiene strings que representan floats
production_countries = production_countries[~production_countries['production_countries'].apply(is_float_string)]

In [34]:
production_countries_df = pd.DataFrame(production_countries_list_dict)

In [35]:
production_countries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49423 entries, 0 to 49422
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   iso_3166_1  49423 non-null  object
 1   name        49423 non-null  object
 2   movie_id    49423 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [39]:
production_countries_df.to_parquet('../data/data_processed/production_countries_ETL.parquet')